In [75]:
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch
import numpy as np
from sklearn.ensemble import RandomForestClassifier

tokens = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

model_sen = SentenceTransformer('all-MiniLM-L6-v2')

def embd_vector1(x):
    tok_id = tokens(x, padding = True, truncation = True, return_tensors = 'pt')

    with torch.no_grad():
        output = model(**tok_id)
    
   # las_embed = output.last_hidden_state
    #cls_embeddings = las_embed[:, 0, :] 

    #vec = model(**tok_id).last_hidden_item
    return output.last_hidden_state.mean(dim=1).squeeze().tolist()

def embd_vector(text):
    return model_sen.encode(text, convert_to_numpy=True).tolist()


def join_text(lt):
    return " ".join(lt)


DF_SOURCE1 = pd.read_csv(r"D:\PYTHON\LLM_CLASSIFICATION_KAGGLE\Dataset\llm-classification-finetuning\train.csv")

def DF_Preprocess(DF_SOURCE):


    x = DF_SOURCE[['prompt', 'response_a', 'response_b']]


    x['consolidated'] = "Prompt: " + x['prompt'] + "response_a: " + x['response_a'] + "response_b: " + x['response_b']


    x.drop(columns=['prompt', 'response_a', 'response_b'], inplace = True)



    y = torch.tensor(DF_SOURCE[['winner_model_a', 'winner_model_b', 'winner_tie']].values)
    y = torch.argmax(y, dim=1).tolist()
    y_df = pd.DataFrame(y)
    for i in x.columns:
        x[i] = x[i].apply(embd_vector)
        


    DF_EVAL = pd.concat([x,y_df], axis=1)
    DF_EVAL.rename(columns={0 : "Actual"}, inplace= True)
    return DF_EVAL

DF_EVAL =  DF_Preprocess(DF_SOURCE1)
DF_EVAL.to_csv("Embed_train.csv")





C:\Users\VINOTH\AppData\Local\Temp\ipykernel_10288\966064847.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['consolidated'] = "Prompt: " + x['prompt'] + "response_a: " + x['response_a'] + "response_b: " + x['response_b']
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_10288\966064847.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.drop(columns=['prompt', 'response_a', 'response_b'], inplace = True)
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_10288\966064847.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ast


DF_TRAIN = pd.read_csv("D:\PYTHON\LLM_CLASSIFICATION_KAGGLE\Embed_train.csv")

print(DF_TRAIN.columns)

DF_TRAIN['consolidated'] = DF_TRAIN['consolidated'].apply(ast.literal_eval)

DF_IND = pd.DataFrame(DF_TRAIN['consolidated'].to_list())

DF_IND.to_csv("df_ind.csv")

DF_IND.head()




<>:10: SyntaxWarning: invalid escape sequence '\P'
<>:10: SyntaxWarning: invalid escape sequence '\P'
C:\Users\VINOTH\AppData\Local\Temp\ipykernel_28432\3757302788.py:10: SyntaxWarning: invalid escape sequence '\P'
  DF_TRAIN = pd.read_csv("D:\PYTHON\LLM_CLASSIFICATION_KAGGLE\Embed_train.csv")


Index(['Unnamed: 0', 'consolidated', 'Actual'], dtype='object')


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.025669,0.005369,-0.038254,0.014313,-0.040225,0.065503,-0.036204,-0.058132,-0.046955,-0.000286,...,-0.018506,0.032707,0.006812,-0.009779,-0.037301,0.042187,0.095915,0.005455,-0.050981,-0.029615
1,-0.006360,0.059760,0.023489,0.023992,-0.085984,0.091230,0.020206,-0.004656,0.069498,-0.040986,...,-0.100965,0.068610,0.082808,0.010535,0.062725,0.085089,-0.014561,0.077452,0.020137,-0.017614
2,-0.085274,0.036727,0.002647,-0.010576,-0.138973,-0.127312,0.059594,0.111553,0.010960,-0.033244,...,-0.003343,0.069076,-0.033395,-0.019938,-0.006622,0.079040,-0.003298,0.159203,0.083756,-0.055680
3,-0.068364,-0.020779,-0.043004,0.027835,0.000839,-0.024870,-0.019578,0.024570,-0.050910,-0.064350,...,-0.030212,0.054888,0.013891,-0.004943,-0.023017,0.020383,0.176729,0.080034,-0.096659,0.030012
4,0.084832,0.114480,0.017523,-0.043089,-0.062486,-0.011723,0.072811,0.042801,-0.048233,0.058617,...,0.057627,0.012610,-0.094707,-0.113518,0.001864,0.133359,0.074227,0.009441,-0.094240,-0.048816


In [4]:
DF_IND['Actual'] = DF_TRAIN['Actual']
DF_IND.head()

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,Actual
0,0.025669,0.005369,-0.038254,0.014313,-0.040225,0.065503,-0.036204,-0.058132,-0.046955,-0.000286,...,0.032707,0.006812,-0.009779,-0.037301,0.042187,0.095915,0.005455,-0.050981,-0.029615,0
1,-0.006360,0.059760,0.023489,0.023992,-0.085984,0.091230,0.020206,-0.004656,0.069498,-0.040986,...,0.068610,0.082808,0.010535,0.062725,0.085089,-0.014561,0.077452,0.020137,-0.017614,1
2,-0.085274,0.036727,0.002647,-0.010576,-0.138973,-0.127312,0.059594,0.111553,0.010960,-0.033244,...,0.069076,-0.033395,-0.019938,-0.006622,0.079040,-0.003298,0.159203,0.083756,-0.055680,2
3,-0.068364,-0.020779,-0.043004,0.027835,0.000839,-0.024870,-0.019578,0.024570,-0.050910,-0.064350,...,0.054888,0.013891,-0.004943,-0.023017,0.020383,0.176729,0.080034,-0.096659,0.030012,0
4,0.084832,0.114480,0.017523,-0.043089,-0.062486,-0.011723,0.072811,0.042801,-0.048233,0.058617,...,0.012610,-0.094707,-0.113518,0.001864,0.133359,0.074227,0.009441,-0.094240,-0.048816,1


In [ ]:
def measure(x,y):
    print(f"classification report:  {classification_report(x,y)}" )
    print(f"confusion_matrix report:  {confusion_matrix(x,y)}" )
    print(f"F1_score : {f1_score(x,y)}")


x_trad = DF_IND.drop(columns=['Actual'])
y_trad = DF_IND['Actual']



x_train, x_test, y_train, y_test = train_test_split(x_trad, y_trad, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(x_train, y_train)
prd_train = model.predict(x_train)
prd_test = model.predict(x_test)
prd_train_prob = model.predict_proba(x_train)


print(f"classification report train:  {classification_report(prd_train,y_train)}")

print(f"classification report test:  {classification_report(prd_test,y_test)}")









classification report train:                precision    recall  f1-score   support

           0       0.92      0.76      0.83     19415
           1       0.89      0.84      0.86     16537
           2       0.57      0.81      0.67     10029

    accuracy                           0.80     45981
   macro avg       0.79      0.80      0.79     45981
weighted avg       0.83      0.80      0.81     45981

classification report test:                precision    recall  f1-score   support

           0       0.51      0.37      0.43      5559
           1       0.35      0.36      0.35      3814
           2       0.28      0.47      0.35      2123

    accuracy                           0.38     11496
   macro avg       0.38      0.40      0.38     11496
weighted avg       0.41      0.38      0.39     11496



In [16]:
from sklearn.linear_model import LogisticRegression

def measure(x,y):
    print(f"classification report:  {classification_report(x,y)}" )
    print(f"confusion_matrix report:  {confusion_matrix(x,y)}" )
    print(f"F1_score : {f1_score(x,y)}")


x_trad = DF_IND.drop(columns=['Actual'])
y_trad = DF_IND['Actual']



x_train, x_test, y_train, y_test = train_test_split(x_trad, y_trad, test_size=0.3, random_state=42)

model1 = LogisticRegression()
model1.fit(x_train, y_train)
prd_train = model1.predict(x_train)
prd_test = model1.predict(x_test)
prd_train_prob = model1.predict_proba(x_train)


print(f"classification report train:  {classification_report(prd_train,y_train)}")

print(f"classification report test:  {classification_report(prd_test,y_test)}")

print(f"f1_score train: {f1_score(prd_train,y_train, average='macro')}")
print(f"f1_score test: {f1_score(prd_test,y_test, average='macro')}")

classification report train:                precision    recall  f1-score   support

           0       0.46      0.41      0.43     15708
           1       0.40      0.41      0.41     13601
           2       0.38      0.43      0.40     10924

    accuracy                           0.41     40233
   macro avg       0.41      0.42      0.41     40233
weighted avg       0.42      0.41      0.42     40233

classification report test:                precision    recall  f1-score   support

           0       0.43      0.38      0.40      6739
           1       0.36      0.37      0.36      5744
           2       0.37      0.41      0.39      4761

    accuracy                           0.39     17244
   macro avg       0.38      0.39      0.38     17244
weighted avg       0.39      0.39      0.39     17244

f1_score train: 0.4131778806961767
f1_score test: 0.38493772208305604


In [17]:
from sklearn.tree import DecisionTreeClassifier

def measure(x,y):
    print(f"classification report:  {classification_report(x,y)}" )
    print(f"confusion_matrix report:  {confusion_matrix(x,y)}" )
    print(f"F1_score : {f1_score(x,y)}")


x_trad = DF_IND.drop(columns=['Actual'])
y_trad = DF_IND['Actual']



x_train, x_test, y_train, y_test = train_test_split(x_trad, y_trad, test_size=0.3, random_state=42)

model1 = DecisionTreeClassifier()
model1.fit(x_train, y_train)
prd_train = model1.predict(x_train)
prd_test = model1.predict(x_test)
prd_train_prob = model1.predict_proba(x_train)


print(f"classification report train:  {classification_report(prd_train,y_train)}")

print(f"classification report test:  {classification_report(prd_test,y_test)}")

print(f"f1_score train: {f1_score(prd_train,y_train, average='macro')}")
print(f"f1_score test: {f1_score(prd_test,y_test, average='macro')}")

classification report train:                precision    recall  f1-score   support

           0       1.00      0.99      0.99     14102
           1       0.99      0.99      0.99     13715
           2       0.99      0.99      0.99     12416

    accuracy                           0.99     40233
   macro avg       0.99      0.99      0.99     40233
weighted avg       0.99      0.99      0.99     40233

classification report test:                precision    recall  f1-score   support

           0       0.36      0.36      0.36      6053
           1       0.36      0.35      0.35      5927
           2       0.34      0.34      0.34      5264

    accuracy                           0.35     17244
   macro avg       0.35      0.35      0.35     17244
weighted avg       0.35      0.35      0.35     17244

f1_score train: 0.9914568146451465
f1_score test: 0.35061681544310835


In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Define parameter grid
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize base model
dtree = DecisionTreeClassifier(random_state=42)

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=dtree,
                           param_grid=param_grid,
                           scoring='f1_macro',
                           cv=5,
                           n_jobs=-1,
                           verbose=1)

# Fit
grid_search.fit(x_train, y_train)

# Best model
best_model = grid_search.best_estimator_

# Predictions
prd_train = best_model.predict(x_train)
prd_test = best_model.predict(x_test)

# Evaluation
print(f"\nBest Parameters: {grid_search.best_params_}")
print(f"Classification Report (Train):\n{classification_report(y_train, prd_train)}")
print(f"Classification Report (Test):\n{classification_report(y_test, prd_test)}")
print(f"F1 Score (Train): {f1_score(y_train, prd_train, average='macro')}")
print(f"F1 Score (Test): {f1_score(y_test, prd_test, average='macro')}")

Fitting 5 folds for each of 135 candidates, totalling 675 fits


c:\Users\VINOTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,



Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
Classification Report (Train):
              precision    recall  f1-score   support

           0       0.38      0.55      0.45     13997
           1       0.38      0.30      0.34     13796
           2       0.41      0.31      0.35     12440

    accuracy                           0.39     40233
   macro avg       0.39      0.39      0.38     40233
weighted avg       0.39      0.39      0.38     40233

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.37      0.52      0.43      6067
           1       0.35      0.28      0.31      5856
           2       0.38      0.30      0.33      5321

    accuracy                           0.37     17244
   macro avg       0.37      0.36      0.36     17244
weighted avg       0.37      0.37      0.36     17244

F1 Score (Train): 0.38087363816009295
F1 Score (Test): 0.3591380357354195


In [49]:
x_train

tensor([[ 0.0152,  0.0427,  0.0751,  ...,  0.0516,  0.0212,  0.0478],
        [-0.0377,  0.0261, -0.0587,  ...,  0.1451, -0.0388,  0.0232],
        [-0.0167,  0.0114, -0.0353,  ...,  0.0840, -0.0945,  0.0359],
        ...,
        [ 0.0478, -0.0097,  0.0756,  ..., -0.0612, -0.0925, -0.0035],
        [-0.0745,  0.0543, -0.0065,  ..., -0.0451, -0.0848,  0.0053],
        [-0.0362,  0.0067,  0.0308,  ...,  0.1293, -0.1251, -0.0109]])

In [ ]:
from torch import nn
from torch import optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import ast

DF_NN = pd.read_csv(r"D:\PYTHON\LLM_CLASSIFICATION_KAGGLE\Embed_train.csv")
DF_NN["consolidated"] = DF_NN['consolidated'].apply(ast.literal_eval)
DF_NN_SPLIT = pd.DataFrame(DF_NN["consolidated"].values.tolist())



In [20]:
import torch
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

DF_NN_SPLIT.columns = DF_NN_SPLIT.columns.astype(str)

scaler_array = scaler.fit_transform(DF_NN_SPLIT) 

DF_NN_SPLIT_1 = pd.DataFrame(scaler_array, columns=DF_NN_SPLIT.columns)

print(DF_NN_SPLIT_1.head())

DF_NN_SPLIT_1['Actual'] = DF_NN['Actual']
X_NN = DF_NN_SPLIT_1.drop(['Actual'], axis=1)
Y_NN = DF_NN_SPLIT_1['Actual']

X_NN_TENSOR = torch.tensor(X_NN.values, dtype= torch.float32)
Y_NN_TENSOR = torch.tensor(Y_NN.values, dtype= torch.long)




          0         1         2         3         4         5         6  \
0  1.062108 -0.478341 -0.768869  0.570151 -0.764261  1.351149 -1.565243   
1  0.375476  0.547580  0.507316  0.791795 -1.665290  1.910902 -0.500860   
2 -1.316271  0.113138  0.076527  0.000162 -2.708699 -2.843962  0.242341   
3 -0.953757 -0.971526 -0.867068  0.879823  0.044311 -0.615116 -1.251534   
4  2.330442  1.579696  0.384004 -0.744421 -1.202598 -0.329069  0.491719   

          7         8         9  ...       375       376       377       378  \
0 -1.955132 -0.515351  0.119071  ...  0.219746  0.324032 -0.343080 -0.795688   
1 -0.814805  1.919156 -0.796727  ...  0.945445  2.042512  0.043316  1.329470   
2  1.663213  0.695383 -0.622522  ...  0.954848 -0.585170 -0.536318 -0.143887   
3 -0.191594 -0.598018 -1.322426  ...  0.668072  0.484109 -0.251096 -0.492214   
4  0.197162 -0.542063  1.444441  ... -0.186465 -1.971621 -2.316373  0.036407   

        379       380       381       382       383    Actual  
0  0

In [21]:
Dataset = TensorDataset(X_NN_TENSOR, Y_NN_TENSOR)
Dataloader = DataLoader(Dataset, batch_size = 32, shuffle = True)
print(X_NN_TENSOR.shape[1])

384


In [28]:
class llmnn(nn.Module):
    def __init__(self):
        super(llmnn, self).__init__()
        self.fc1 = nn.Linear(X_NN_TENSOR.shape[1], 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 16)
        self.fc4 = nn.Linear(16, 3)
        self.leakyrelu = nn.LeakyReLU()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        
        return x

module = llmnn()
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(module.parameters(),  lr=0.001)

for epoch in range(45):
    correct = 0
    total_loss = 0
    for x,y in Dataloader:
        opt.zero_grad()
        output = module(x)
        loss = loss_fn(output,y)
        loss.backward()
        opt.step()
        prd = torch.argmax(output,dim=1)
        total_loss += loss.item() 
        correct += (prd==y).sum().item()

    acc = correct / len(Dataloader.dataset)
    print(f"Epoch {epoch+1}: Loss = {loss:.4f}, Accuracy = {acc:.4f}")

torch.save(module.state_dict(), "llmnn_model.pth")
print("Model saved to llmnn_model.pth")












Epoch 1: Loss = 1.1236, Accuracy = 0.3778
Epoch 2: Loss = 1.0113, Accuracy = 0.3975
Epoch 3: Loss = 1.0462, Accuracy = 0.4125
Epoch 4: Loss = 1.0714, Accuracy = 0.4304
Epoch 5: Loss = 1.1270, Accuracy = 0.4499
Epoch 6: Loss = 0.9436, Accuracy = 0.4786
Epoch 7: Loss = 0.8110, Accuracy = 0.5092
Epoch 8: Loss = 0.8462, Accuracy = 0.5402
Epoch 9: Loss = 0.5772, Accuracy = 0.5711
Epoch 10: Loss = 0.4837, Accuracy = 0.5999
Epoch 11: Loss = 1.5735, Accuracy = 0.6234
Epoch 12: Loss = 0.5984, Accuracy = 0.6488
Epoch 13: Loss = 0.4488, Accuracy = 0.6692
Epoch 14: Loss = 1.2967, Accuracy = 0.6868
Epoch 15: Loss = 0.7547, Accuracy = 0.7014
Epoch 16: Loss = 0.7344, Accuracy = 0.7198
Epoch 17: Loss = 0.7666, Accuracy = 0.7335
Epoch 18: Loss = 0.2154, Accuracy = 0.7456
Epoch 19: Loss = 0.3769, Accuracy = 0.7567
Epoch 20: Loss = 0.6123, Accuracy = 0.7659
Epoch 21: Loss = 0.7606, Accuracy = 0.7746
Epoch 22: Loss = 0.2201, Accuracy = 0.7857
Epoch 23: Loss = 0.4476, Accuracy = 0.7922
Epoch 24: Loss = 0.7

In [40]:
from sentence_transformers import SentenceTransformer

model_sen = SentenceTransformer('all-MiniLM-L6-v2')

def embd_vector(text):
    return model_sen.encode(text, convert_to_numpy=True).tolist()


def DF_Preprocess1(DF_SOURCE):


    x = DF_SOURCE[['prompt', 'response_a', 'response_b']]


    x['consolidated'] = "Prompt: " + x['prompt'] + "response_a: " + x['response_a'] + "response_b: " + x['response_b']


    x.drop(columns=['prompt', 'response_a', 'response_b'], inplace = True)



   
    for i in x.columns:
        x[i] = x[i].apply(embd_vector)

    return x



In [43]:
DF_TEST_EVAL = pd.read_csv(r"D:\PYTHON\LLM_CLASSIFICATION_KAGGLE\Dataset\llm-classification-finetuning\test.csv")

DF_TEST_EVAL
#DF_TEST_EMBED = DF_TEST_EVAL.drop( columns = ['winner_model_a', 'winner_model_b', 'winner_tie'])

DF_TEST_EMBED = DF_Preprocess1(DF_TEST_EVAL)
DF_TEST_EMBED

,consolidated
0,"[0.0434182733297348, 0.1418384164571762, 3.547..."
1,"[-0.010802901349961758, 0.04609622061252594, -..."
2,"[-0.04522497206926346, -0.08193384110927582, 0..."


In [57]:
import ast
DF_TEST = pd.DataFrame(DF_TEST_EMBED["consolidated"].values.tolist())

DF_TEST.columns = DF_TEST.columns.astype(str)

scaler_array_test = scaler.fit_transform(DF_TEST) 

DF_TEST1 = pd.DataFrame(scaler_array_test, columns=DF_TEST.columns)




X_test = torch.tensor(DF_TEST1.values, dtype= torch.float32)
module.eval()

with torch.no_grad():
    output = module(X_test)
    predicted_class = nn.functional.softmax(output)
    print(predicted_class)


DF_TEST_Y = pd.DataFrame(predicted_class, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
DF_TEST_Y ["id"] = DF_TEST_EVAL["id"]
cols = ['id'] + [col for col in DF_TEST_Y.columns if col != 'id']
DF_TEST_Y1 = DF_TEST_Y[cols]
DF_TEST_Y1.to_csv("submit.csv")





tensor([[0.2560, 0.7430, 0.0010],
        [0.0215, 0.3575, 0.6210],
        [0.2135, 0.7328, 0.0537]])


C:\Users\VINOTH\AppData\Local\Temp\ipykernel_31424\2264010219.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predicted_class = nn.functional.softmax(output)
